In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from IPython.core.display import display, HTML
import re
import dateutil.parser
from sklearn.linear_model import LinearRegression
import seaborn as sns

In [2]:
#create an empty data frame with all the column names you want
#try find_all with text = player name

In [3]:
#create a way to interate through all pages I will need to scrape

main_df = pd.DataFrame(columns = ['SALARY','PLAYER','POS','MPG','PPG','RPG','APG','TPG','2PTFG_PERC','3PTFG_PERC',
                       'FT_PERC','SPG','BPG'])

nba_teams = ['PHI','BRK','MIL','NYK','ATL','MIA','BOS','WAS','IND','CHO','CHI','TOR','CLE','ORL','DET',
            'UTA','PHO','DEN','LAC','DAL','POR','LAL','MEM','GSW','SAS','NOP','SAC','MIN','OKC','HOU']

years = ['2021','2020','2019','2018','2017','2016','2015','2014','2013']

base_url = 'https://www.basketball-reference.com/teams/'


link_list= []
for team in nba_teams:
    for year in years:
        new_url = base_url + team +"/"+ year +".html"
        link_list.append(new_url)


In [4]:
#let's start with trying to scrape one page using GSW 2020

url = 'https://www.basketball-reference.com/teams/GSW/2020.html'
response = requests.get(url)

In [5]:
response.status_code

200

In [6]:
response.text[:1000]

'\n<!DOCTYPE html>\n<html data-version="klecko-" data-root="/home/bbr/build" itemscope itemtype="https://schema.org/WebSite" lang="en" class="no-js" >\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />\n    <link rel="dns-prefetch" href="https://d2p3bygnnzw9w3.cloudfront.net/req/202107281" />\n<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->\n<script type="text/javascript" async=true>\n    (function() {\n\tvar host = window.location.hostname;\n\tvar element = document.createElement(\'script\');\n\tvar firstScript = document.getElementsByTagName(\'script\')[0];\n\tvar url = \'https://quantcast.mgr.consensu.org\'\n\t    .concat(\'/choice/\', \'XwNYEpNeFfhfr\', \'/\', host, \'/choice.js\')\n\tvar uspTries = 0;\n\tvar uspTriesLimit = 3;\n\telement.async = true;\n\telement.type = \'text/javascript\';\n\telement.src = url;\n\t\n\tfirstS

In [7]:
gsw_page = response.text

In [8]:
soup = BeautifulSoup(gsw_page,'lxml')

In [9]:
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" data-root="/home/bbr/build" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
  <link href="https://d2p3bygnnzw9w3.cloudfront.net/req/202107281" rel="dns-prefetch"/>
  <!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
  <script async="true" type="text/javascript">
   (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://quantcast.mgr.consensu.org'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, '/choice.js')
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);

In [10]:
#get results from the roster table


roster_results = soup.find('div', id = 'div_roster').find_all('tr')

roster_dict={}

for player in roster_results[1:]:
    roster_data = player.find_all('td')
    roster_player = roster_data[0].text
    roster_dict[roster_player]=[i.text for i in roster_data]
roster_dict


    
    
    
    
    
    

{'Dragan Bender': ['Dragan Bender',
  'PF',
  '7-0',
  '225',
  'November 17, 1997',
  'ba',
  '3',
  ''],
 'Ky Bowman': ['Ky Bowman',
  'PG',
  '6-1',
  '187',
  'June 16, 1997',
  'us',
  'R',
  'Boston College'],
 'Alec Burks': ['Alec Burks',
  'SF',
  '6-6',
  '214',
  'July 20, 1991',
  'us',
  '8',
  'Colorado'],
 'Willie Cauley-Stein': ['Willie Cauley-Stein',
  'C',
  '7-0',
  '240',
  'August 18, 1993',
  'us',
  '4',
  'Kentucky'],
 'Marquese Chriss': ['Marquese Chriss',
  'PF',
  '6-9',
  '240',
  'July 2, 1997',
  'us',
  '3',
  'Washington'],
 'Stephen Curry': ['Stephen Curry',
  'PG',
  '6-3',
  '185',
  'March 14, 1988',
  'us',
  '10',
  'Davidson'],
 'Jacob Evans': ['Jacob Evans',
  'SG',
  '6-4',
  '210',
  'June 18, 1997',
  'us',
  '1',
  'Cincinnati'],
 'Draymond Green': ['Draymond Green',
  'PF',
  '6-6',
  '230',
  'March 4, 1990',
  'us',
  '7',
  'Michigan State'],
 'Damion Lee': ['Damion Lee',
  'SG',
  '6-5',
  '210',
  'October 21, 1992',
  'us',
  '2',
  'Dr

In [11]:
roster_df = pd.DataFrame(roster_dict).T
roster_df.columns = ['PLAYER','POSITION','HEIGHT','WEIGHT','DOB','COUNTRY','EXPERIENCE','COLLEGE']
roster_df

,PLAYER,POSITION,HEIGHT,WEIGHT,DOB,COUNTRY,EXPERIENCE,COLLEGE
Dragan Bender,Dragan Bender,PF,7-0,225,"November 17, 1997",ba,3,
Ky Bowman,Ky Bowman,PG,6-1,187,"June 16, 1997",us,R,Boston College
Alec Burks,Alec Burks,SF,6-6,214,"July 20, 1991",us,8,Colorado
Willie Cauley-Stein,Willie Cauley-Stein,C,7-0,240,"August 18, 1993",us,4,Kentucky
Marquese Chriss,Marquese Chriss,PF,6-9,240,"July 2, 1997",us,3,Washington
Stephen Curry,Stephen Curry,PG,6-3,185,"March 14, 1988",us,10,Davidson
Jacob Evans,Jacob Evans,SG,6-4,210,"June 18, 1997",us,1,Cincinnati
Draymond Green,Draymond Green,PF,6-6,230,"March 4, 1990",us,7,Michigan State
Damion Lee,Damion Lee,SG,6-5,210,"October 21, 1992",us,2,"Drexel, Louisville"
Kevon Looney,Kevon Looney,C,6-9,222,"February 6, 1996",us,4,UCLA


In [12]:
pergame_results = soup.find('div', id = 'div_per_game').find_all('tr')

pergame_dict={}

for player in pergame_results[1:]:
    pergame_data = player.find_all('td')
    pergame_player = pergame_data[0].text
    pergame_dict[pergame_player]=[i.text for i in pergame_data]
pergame_dict


{'Andrew Wiggins': ['Andrew Wiggins',
  '24',
  '12',
  '12',
  '33.6',
  '7.2',
  '15.7',
  '.457',
  '1.7',
  '4.9',
  '.339',
  '5.5',
  '10.8',
  '.512',
  '.511',
  '3.4',
  '5.1',
  '.672',
  '1.5',
  '3.1',
  '4.6',
  '3.6',
  '1.3',
  '1.4',
  '2.1',
  '2.2',
  '19.4'],
 "D'Angelo Russell": ["D'Angelo Russell",
  '23',
  '33',
  '33',
  '32.1',
  '8.3',
  '19.2',
  '.430',
  '3.6',
  '9.7',
  '.374',
  '4.6',
  '9.5',
  '.487',
  '.524',
  '3.4',
  '4.4',
  '.785',
  '0.4',
  '3.3',
  '3.7',
  '6.2',
  '0.9',
  '0.3',
  '3.1',
  '1.9',
  '23.6'],
 'Glenn Robinson III': ['Glenn Robinson III',
  '26',
  '48',
  '48',
  '31.6',
  '5.1',
  '10.6',
  '.481',
  '1.4',
  '3.5',
  '.400',
  '3.7',
  '7.0',
  '.522',
  '.548',
  '1.3',
  '1.5',
  '.851',
  '1.4',
  '3.3',
  '4.7',
  '1.8',
  '0.9',
  '0.3',
  '1.0',
  '1.7',
  '12.9'],
 'Mychal Mulder': ['Mychal Mulder',
  '25',
  '7',
  '3',
  '29.1',
  '3.7',
  '9.6',
  '.388',
  '2.3',
  '7.4',
  '.308',
  '1.4',
  '2.1',
  '.667',
 

In [13]:
pergame_df = pd.DataFrame(pergame_dict).T
pergame_df.columns = ['PLAYER','AGE','GAMES','GAMES_STARTED','MINUTES_PLAYED','FIELD_GOALS','FIELD_GOALS_ATT',
                      'FIELD_GOAL_PCT','3PT_M','3PTS_A','3PT_PCT','2PT_M','2PT_A','2PT_PCT','EFG_PCT','FT_M',
                     'FT_A','FT_PCT','OFF_REB','DEF_REB','TOT_REB','ASSISTS','STEALS','BLOCKS','TO','PF','POINTS']
pergame_df

,PLAYER,AGE,GAMES,GAMES_STARTED,MINUTES_PLAYED,FIELD_GOALS,FIELD_GOALS_ATT,FIELD_GOAL_PCT,3PT_M,3PTS_A,...,FT_PCT,OFF_REB,DEF_REB,TOT_REB,ASSISTS,STEALS,BLOCKS,TO,PF,POINTS
Andrew Wiggins,Andrew Wiggins,24,12,12,33.6,7.2,15.7,.457,1.7,4.9,...,.672,1.5,3.1,4.6,3.6,1.3,1.4,2.1,2.2,19.4
D'Angelo Russell,D'Angelo Russell,23,33,33,32.1,8.3,19.2,.430,3.6,9.7,...,.785,0.4,3.3,3.7,6.2,0.9,0.3,3.1,1.9,23.6
Glenn Robinson III,Glenn Robinson III,26,48,48,31.6,5.1,10.6,.481,1.4,3.5,...,.851,1.4,3.3,4.7,1.8,0.9,0.3,1.0,1.7,12.9
Mychal Mulder,Mychal Mulder,25,7,3,29.1,3.7,9.6,.388,2.3,7.4,...,.750,0.4,2.9,3.3,1.1,0.3,0.1,0.7,2.9,11.0
Alec Burks,Alec Burks,28,48,18,29.0,5.1,12.5,.406,1.8,4.7,...,.897,0.8,3.9,4.7,3.1,1.0,0.4,1.6,2.2,16.1
Damion Lee,Damion Lee,27,49,36,29.0,4.3,10.4,.417,1.7,4.9,...,.873,0.7,4.2,4.9,2.7,1.0,0.1,1.4,2.4,12.7
Draymond Green,Draymond Green,29,43,43,28.4,2.9,7.3,.389,0.8,3.0,...,.759,0.5,5.7,6.2,6.2,1.4,0.8,2.3,2.6,8.0
Stephen Curry,Stephen Curry,31,5,5,27.8,6.6,16.4,.402,2.4,9.8,...,1.000,0.8,4.4,5.2,6.6,1.0,0.4,3.2,2.2,20.8
Eric Paschall,Eric Paschall,23,60,26,27.6,5.3,10.7,.497,0.6,2.2,...,.774,1.4,3.2,4.6,2.1,0.5,0.2,1.6,2.1,14.0
Willie Cauley-Stein,Willie Cauley-Stein,26,41,37,22.9,3.5,6.3,.560,0.0,0.0,...,.614,1.7,4.5,6.2,1.5,1.1,1.2,0.9,2.5,7.9


In [14]:
#create a function to gather roster data from every page 
def get_roster_data(url):
    response = requests.get(url)
    web_page = response.text
    soup = BeautifulSoup(web_page,'lxml')
    try: 
        roster_results = soup.find('div', id = 'div_roster').find_all('tr')
    except AttributeError:
        return pd.DataFrame()
    roster_dict={}
    for player in roster_results[1:]:
        roster_data = player.find_all('td')
        roster_player = roster_data[0].text
        roster_dict[roster_player]=[i.text for i in roster_data]
    roster_df = pd.DataFrame(roster_dict).T
    roster_df.columns = ['PLAYER','POSITION','HEIGHT','WEIGHT','DOB','COUNTRY','EXPERIENCE','COLLEGE']
    roster_df['YEAR']= url[-9:-5]
    roster_df['TEAM']=url[-13:-10]
    return roster_df


    

In [15]:
#get roster data for every link we want to use

roster_df_list = []
for link in link_list:
    new_df = get_roster_data(link)
    roster_df_list.append(new_df)
roster_df_complete = pd.concat(roster_df_list)

# roster_df_complete


In [16]:
roster_df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4983 entries, Danny Green to Greg Smith
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PLAYER      4983 non-null   object
 1   POSITION    4983 non-null   object
 2   HEIGHT      4983 non-null   object
 3   WEIGHT      4983 non-null   object
 4   DOB         4983 non-null   object
 5   COUNTRY     4983 non-null   object
 6   EXPERIENCE  4983 non-null   object
 7   COLLEGE     4983 non-null   object
 8   YEAR        4983 non-null   object
 9   TEAM        4983 non-null   object
dtypes: object(10)
memory usage: 428.2+ KB


In [17]:
#create function to collect per game data
#need to account for year year = str[-9:-5]
#https://www.basketball-reference.com/teams/GSW/2020.html

def get_pergame_data(url):
    response = requests.get(url)
    web_page = response.text
    soup = BeautifulSoup(web_page,'lxml')
    try:
        pergame_results = soup.find('div', id = 'div_per_game').find_all('tr')
    except AttributeError:
        return pd.DataFrame()
    pergame_dict={}
    for player in pergame_results[1:]:
        pergame_data = player.find_all('td')
        pergame_player = pergame_data[0].text
        pergame_dict[pergame_player]=[i.text for i in pergame_data]
    pergame_df = pd.DataFrame(pergame_dict).T
    pergame_df.columns = ['PLAYER','AGE','GAMES','GAMES_STARTED','MINUTES_PLAYED','FIELD_GOALS','FIELD_GOALS_ATT',
                      'FIELD_GOAL_PCT','3PT_M','3PTS_A','3PT_PCT','2PT_M','2PT_A','2PT_PCT','EFG_PCT','FT_M',
                     'FT_A','FT_PCT','OFF_REB','DEF_REB','TOT_REB','ASSISTS','STEALS','BLOCKS','TO','PF','POINTS']
    pergame_df['YEAR']= url[-9:-5]
    pergame_df['TEAM']=url[-13:-10]
    return pergame_df

In [18]:
#get pergame data for every link we want to use

pergame_df_list = []
for link in link_list:
    new_pergame_df = get_pergame_data(link)
    pergame_df_list.append(new_pergame_df)
pergame_df_list
pergame_df_complete = pd.concat(pergame_df_list)

pergame_df_complete

,PLAYER,AGE,GAMES,GAMES_STARTED,MINUTES_PLAYED,FIELD_GOALS,FIELD_GOALS_ATT,FIELD_GOAL_PCT,3PT_M,3PTS_A,...,DEF_REB,TOT_REB,ASSISTS,STEALS,BLOCKS,TO,PF,POINTS,YEAR,TEAM
Tobias Harris,Tobias Harris,28,62,62,32.5,7.6,14.9,.512,1.3,3.4,...,5.8,6.8,3.5,0.9,0.8,1.7,1.9,19.5,2021,PHI
Ben Simmons,Ben Simmons,24,58,58,32.4,5.6,10.1,.557,0.1,0.2,...,5.6,7.2,6.9,1.6,0.6,3.0,2.9,14.3,2021,PHI
Joel Embiid,Joel Embiid,26,51,51,31.1,9.0,17.6,.513,1.1,3.0,...,8.4,10.6,2.8,1.0,1.4,3.1,2.4,28.5,2021,PHI
Seth Curry,Seth Curry,30,57,57,28.7,4.5,9.7,.467,2.2,4.9,...,2.2,2.4,2.7,0.8,0.1,1.1,1.7,12.5,2021,PHI
Danny Green,Danny Green,33,69,69,28.0,3.2,7.9,.412,2.5,6.3,...,3.0,3.8,1.7,1.3,0.8,1.0,1.8,9.5,2021,PHI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Daequan Cook,Daequan Cook,25,16,1,10.3,1.3,3.7,.356,0.7,1.9,...,0.8,1.1,0.6,0.4,0.1,0.4,0.8,3.4,2013,HOU
Cole Aldrich,Cole Aldrich,24,30,0,7.1,0.8,1.4,.535,0.0,0.0,...,1.5,1.9,0.2,0.1,0.3,0.5,1.4,1.7,2013,HOU
Aaron Brooks,Aaron Brooks,28,7,0,5.4,0.6,1.9,.308,0.3,1.0,...,0.1,0.3,0.9,0.1,0.4,0.6,0.7,1.4,2013,HOU
Tim Ohlbrecht,Tim Ohlbrecht,24,3,0,4.0,0.3,1.0,.333,0.0,0.0,...,0.3,0.3,0.3,0.3,0.0,1.3,0.3,1.0,2013,HOU


In [19]:
#drop duplicate players on the roster dataframe
# roster_df_complete = roster_df_complete.drop_duplicates(subset=['PLAYER'])
roster_df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4983 entries, Danny Green to Greg Smith
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PLAYER      4983 non-null   object
 1   POSITION    4983 non-null   object
 2   HEIGHT      4983 non-null   object
 3   WEIGHT      4983 non-null   object
 4   DOB         4983 non-null   object
 5   COUNTRY     4983 non-null   object
 6   EXPERIENCE  4983 non-null   object
 7   COLLEGE     4983 non-null   object
 8   YEAR        4983 non-null   object
 9   TEAM        4983 non-null   object
dtypes: object(10)
memory usage: 428.2+ KB


In [20]:
#only want the position and experience from the roster table
roster_df_complete = roster_df_complete.drop(['HEIGHT','WEIGHT','DOB','COUNTRY','COLLEGE'], axis = 1)
roster_df_complete.head()

,PLAYER,POSITION,EXPERIENCE,YEAR,TEAM
Danny Green,Danny Green,SF,11,2021,PHI
Dwight Howard,Dwight Howard,C,16,2021,PHI
Matisse Thybulle,Matisse Thybulle,SG,1,2021,PHI
Shake Milton,Shake Milton,SG,2,2021,PHI
Tobias Harris,Tobias Harris,PF,9,2021,PHI


In [21]:
final_df = pd.merge(pergame_df_complete,roster_df_complete, on=['PLAYER','YEAR'],how = 'left')

In [22]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6185 entries, 0 to 6184
Data columns (total 32 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   PLAYER           6185 non-null   object
 1   AGE              6185 non-null   object
 2   GAMES            6185 non-null   object
 3   GAMES_STARTED    6185 non-null   object
 4   MINUTES_PLAYED   6185 non-null   object
 5   FIELD_GOALS      6185 non-null   object
 6   FIELD_GOALS_ATT  6185 non-null   object
 7   FIELD_GOAL_PCT   6185 non-null   object
 8   3PT_M            6185 non-null   object
 9   3PTS_A           6185 non-null   object
 10  3PT_PCT          6185 non-null   object
 11  2PT_M            6185 non-null   object
 12  2PT_A            6185 non-null   object
 13  2PT_PCT          6185 non-null   object
 14  EFG_PCT          6185 non-null   object
 15  FT_M             6185 non-null   object
 16  FT_A             6185 non-null   object
 17  FT_PCT           6185 non-null   

In [23]:
final_df = final_df.drop(['AGE','GAMES_STARTED','FIELD_GOALS_ATT',
                      '3PTS_A','2PT_A','FT_A','PF','EFG_PCT'],axis=1)

In [24]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6185 entries, 0 to 6184
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   PLAYER          6185 non-null   object
 1   GAMES           6185 non-null   object
 2   MINUTES_PLAYED  6185 non-null   object
 3   FIELD_GOALS     6185 non-null   object
 4   FIELD_GOAL_PCT  6185 non-null   object
 5   3PT_M           6185 non-null   object
 6   3PT_PCT         6185 non-null   object
 7   2PT_M           6185 non-null   object
 8   2PT_PCT         6185 non-null   object
 9   FT_M            6185 non-null   object
 10  FT_PCT          6185 non-null   object
 11  OFF_REB         6185 non-null   object
 12  DEF_REB         6185 non-null   object
 13  TOT_REB         6185 non-null   object
 14  ASSISTS         6185 non-null   object
 15  STEALS          6185 non-null   object
 16  BLOCKS          6185 non-null   object
 17  TO              6185 non-null   object
 18  POINTS  

In [25]:
final_df.head()

,PLAYER,GAMES,MINUTES_PLAYED,FIELD_GOALS,FIELD_GOAL_PCT,3PT_M,3PT_PCT,2PT_M,2PT_PCT,FT_M,...,ASSISTS,STEALS,BLOCKS,TO,POINTS,YEAR,TEAM_x,POSITION,EXPERIENCE,TEAM_y
0,Tobias Harris,62,32.5,7.6,.512,1.3,.394,6.3,.546,2.9,...,3.5,0.9,0.8,1.7,19.5,2021,PHI,PF,9,PHI
1,Ben Simmons,58,32.4,5.6,.557,0.1,.300,5.6,.562,3.0,...,6.9,1.6,0.6,3.0,14.3,2021,PHI,PG,3,PHI
2,Joel Embiid,51,31.1,9.0,.513,1.1,.377,7.9,.541,9.2,...,2.8,1.0,1.4,3.1,28.5,2021,PHI,C,4,PHI
3,Seth Curry,57,28.7,4.5,.467,2.2,.450,2.3,.485,1.2,...,2.7,0.8,0.1,1.1,12.5,2021,PHI,SG,6,PHI
4,Danny Green,69,28.0,3.2,.412,2.5,.405,0.7,.438,0.4,...,1.7,1.3,0.8,1.0,9.5,2021,PHI,SF,11,PHI


In [26]:
final_df.columns

Index(['PLAYER', 'GAMES', 'MINUTES_PLAYED', 'FIELD_GOALS', 'FIELD_GOAL_PCT',
       '3PT_M', '3PT_PCT', '2PT_M', '2PT_PCT', 'FT_M', 'FT_PCT', 'OFF_REB',
       'DEF_REB', 'TOT_REB', 'ASSISTS', 'STEALS', 'BLOCKS', 'TO', 'POINTS',
       'YEAR', 'TEAM_x', 'POSITION', 'EXPERIENCE', 'TEAM_y'],
      dtype='object')

In [27]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6185 entries, 0 to 6184
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   PLAYER          6185 non-null   object
 1   GAMES           6185 non-null   object
 2   MINUTES_PLAYED  6185 non-null   object
 3   FIELD_GOALS     6185 non-null   object
 4   FIELD_GOAL_PCT  6185 non-null   object
 5   3PT_M           6185 non-null   object
 6   3PT_PCT         6185 non-null   object
 7   2PT_M           6185 non-null   object
 8   2PT_PCT         6185 non-null   object
 9   FT_M            6185 non-null   object
 10  FT_PCT          6185 non-null   object
 11  OFF_REB         6185 non-null   object
 12  DEF_REB         6185 non-null   object
 13  TOT_REB         6185 non-null   object
 14  ASSISTS         6185 non-null   object
 15  STEALS          6185 non-null   object
 16  BLOCKS          6185 non-null   object
 17  TO              6185 non-null   object
 18  POINTS  

In [28]:
column_list = list(final_df.columns)
column_list = column_list[2:19]
column_list

final_df[column_list] = final_df[column_list].apply(pd.to_numeric, errors='coerce', axis=1)

In [29]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6185 entries, 0 to 6184
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLAYER          6185 non-null   object 
 1   GAMES           6185 non-null   object 
 2   MINUTES_PLAYED  6185 non-null   float64
 3   FIELD_GOALS     6185 non-null   float64
 4   FIELD_GOAL_PCT  6143 non-null   float64
 5   3PT_M           6185 non-null   float64
 6   3PT_PCT         5547 non-null   float64
 7   2PT_M           6185 non-null   float64
 8   2PT_PCT         6084 non-null   float64
 9   FT_M            6185 non-null   float64
 10  FT_PCT          5815 non-null   float64
 11  OFF_REB         6185 non-null   float64
 12  DEF_REB         6185 non-null   float64
 13  TOT_REB         6185 non-null   float64
 14  ASSISTS         6185 non-null   float64
 15  STEALS          6185 non-null   float64
 16  BLOCKS          6185 non-null   float64
 17  TO              6185 non-null   f

In [30]:
#drop NAs for POS (all low level players with minimal playing time)

final_df.dropna(subset=['POSITION'], inplace = True)

In [31]:
#games need to be int data type

final_df['GAMES'] = final_df['GAMES'].astype('int')

#dropping rookie players since their pay is not based on performance

final_df.drop(final_df[final_df['EXPERIENCE']=='R'].index, inplace=True)

#experience also needs to be an int

final_df['EXPERIENCE'] = final_df['EXPERIENCE'].astype('int')
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5073 entries, 0 to 6181
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLAYER          5073 non-null   object 
 1   GAMES           5073 non-null   int64  
 2   MINUTES_PLAYED  5073 non-null   float64
 3   FIELD_GOALS     5073 non-null   float64
 4   FIELD_GOAL_PCT  5046 non-null   float64
 5   3PT_M           5073 non-null   float64
 6   3PT_PCT         4592 non-null   float64
 7   2PT_M           5073 non-null   float64
 8   2PT_PCT         5003 non-null   float64
 9   FT_M            5073 non-null   float64
 10  FT_PCT          4848 non-null   float64
 11  OFF_REB         5073 non-null   float64
 12  DEF_REB         5073 non-null   float64
 13  TOT_REB         5073 non-null   float64
 14  ASSISTS         5073 non-null   float64
 15  STEALS          5073 non-null   float64
 16  BLOCKS          5073 non-null   float64
 17  TO              5073 non-null   f

In [32]:
 #check NAs for fg pct. all players playes only 1-2 games and little minutes. Will drop them. 
    
final_df[final_df['FIELD_GOAL_PCT'].isna()]

,PLAYER,GAMES,MINUTES_PLAYED,FIELD_GOALS,FIELD_GOAL_PCT,3PT_M,3PT_PCT,2PT_M,2PT_PCT,FT_M,...,ASSISTS,STEALS,BLOCKS,TO,POINTS,YEAR,TEAM_x,POSITION,EXPERIENCE,TEAM_y
366,Rashad Vaughn,1,4.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,1.0,0.0,0.0,0.0,0.0,2018,BRK,SG,2,BRK
367,Rashad Vaughn,1,4.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,1.0,0.0,0.0,0.0,0.0,2018,BRK,SG,2,MIL
368,Rashad Vaughn,1,4.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,1.0,0.0,0.0,0.0,0.0,2018,BRK,SG,2,ORL
479,Damion James,2,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,2013,BRK,SF,2,BRK
1115,Jeremy Tyler,1,5.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,2013,ATL,C,1,ATL
1116,Jeremy Tyler,1,5.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,2013,ATL,C,1,GSW
1246,James Ennis,3,2.3,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.3,0.0,0.0,0.0,0.0,2016,MIA,SF,1,MIA
1247,James Ennis,3,2.3,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.3,0.0,0.0,0.0,0.0,2016,MIA,SF,1,MEM
1248,James Ennis,3,2.3,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.3,0.0,0.0,0.0,0.0,2016,MIA,SF,1,NOP
2102,Tyler Ulis,1,1.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,2019,CHI,PG,2,CHI


In [33]:
#NAs for FG, 3pt,2pt, and ft PCT represent players who did not attempt those types of shots. Will fill with 0
final_df[final_df['3PT_PCT'].isna()]

,PLAYER,GAMES,MINUTES_PLAYED,FIELD_GOALS,FIELD_GOAL_PCT,3PT_M,3PT_PCT,2PT_M,2PT_PCT,FT_M,...,ASSISTS,STEALS,BLOCKS,TO,POINTS,YEAR,TEAM_x,POSITION,EXPERIENCE,TEAM_y
105,Jahlil Okafor,2,12.5,2.0,0.444,0.0,NaN,2.0,0.444,1.0,...,0.5,0.0,1.0,1.5,5.0,2018,PHI,C,2,PHI
106,Jahlil Okafor,2,12.5,2.0,0.444,0.0,NaN,2.0,0.444,1.0,...,0.5,0.0,1.0,1.5,5.0,2018,PHI,C,2,BRK
127,Jahlil Okafor,50,22.7,4.8,0.514,0.0,NaN,4.8,0.514,2.1,...,1.2,0.4,1.0,1.8,11.8,2017,PHI,C,1,PHI
199,JaVale McGee,6,10.2,1.3,0.444,0.0,NaN,1.3,0.444,0.3,...,0.3,0.0,0.2,0.8,3.0,2015,PHI,C,6,PHI
200,JaVale McGee,6,10.2,1.3,0.444,0.0,NaN,1.3,0.444,0.3,...,0.3,0.0,0.2,0.8,3.0,2015,PHI,C,6,DEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6140,Ömer Aşık,48,20.2,2.1,0.532,0.0,NaN,2.1,0.532,1.6,...,0.5,0.3,0.8,1.2,5.8,2014,HOU,C,3,HOU
6142,Josh Powell,1,19.0,2.0,0.333,0.0,NaN,2.0,0.333,0.0,...,0.0,0.0,1.0,1.0,4.0,2014,HOU,PF,6,HOU
6151,Greg Smith,11,9.1,1.6,0.643,0.0,NaN,1.6,0.643,0.2,...,0.0,0.1,0.2,0.5,3.5,2014,HOU,C,2,HOU
6178,Cole Aldrich,30,7.1,0.8,0.535,0.0,NaN,0.8,0.535,0.1,...,0.2,0.1,0.3,0.5,1.7,2013,HOU,C,2,SAC


In [34]:
final_df['3PT_PCT']= final_df['3PT_PCT'].fillna(0)
final_df['2PT_PCT']=final_df['2PT_PCT'].fillna(0)
final_df['FT_PCT']=final_df['FT_PCT'].fillna(0)
final_df['FIELD_GOAL_PCT']=final_df['FIELD_GOAL_PCT'].fillna(0)
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5073 entries, 0 to 6181
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLAYER          5073 non-null   object 
 1   GAMES           5073 non-null   int64  
 2   MINUTES_PLAYED  5073 non-null   float64
 3   FIELD_GOALS     5073 non-null   float64
 4   FIELD_GOAL_PCT  5073 non-null   float64
 5   3PT_M           5073 non-null   float64
 6   3PT_PCT         5073 non-null   float64
 7   2PT_M           5073 non-null   float64
 8   2PT_PCT         5073 non-null   float64
 9   FT_M            5073 non-null   float64
 10  FT_PCT          5073 non-null   float64
 11  OFF_REB         5073 non-null   float64
 12  DEF_REB         5073 non-null   float64
 13  TOT_REB         5073 non-null   float64
 14  ASSISTS         5073 non-null   float64
 15  STEALS          5073 non-null   float64
 16  BLOCKS          5073 non-null   float64
 17  TO              5073 non-null   f

### Grabbing Salary from ESPN

In [35]:
pagenums = np.arange(1,14,1)
url2021_list = []
url2020_list = []
url2019_list = []
url2018_list = []
url2017_list = []
url2016_list = []
url2015_list = []
url2014_list = []
url2013_list = []

for num in pagenums:
    url2021 = 'http://www.espn.com/nba/salaries/_/year/2021/page/'+str(num)
    url2021_list.append(url2021)
    url2020 = 'http://www.espn.com/nba/salaries/_/year/2020/page/'+str(num)
    url2020_list.append(url2020)
    url2019 = 'http://www.espn.com/nba/salaries/_/year/2019/page/'+str(num)
    url2019_list.append(url2019)
    url2018 = 'http://www.espn.com/nba/salaries/_/year/2018/page/'+str(num)
    url2018_list.append(url2018)
    url2017 = 'http://www.espn.com/nba/salaries/_/year/2017/page/'+str(num)
    url2017_list.append(url2017)
    url2016 = 'http://www.espn.com/nba/salaries/_/year/2017/page/'+str(num)
    url2016_list.append(url2016)
    url2015 = 'http://www.espn.com/nba/salaries/_/year/2017/page/'+str(num)
    url2015_list.append(url2015)
    url2014 = 'http://www.espn.com/nba/salaries/_/year/2017/page/'+str(num)
    url2014_list.append(url2014)
    url2013 = 'http://www.espn.com/nba/salaries/_/year/2017/page/'+str(num)
    url2013_list.append(url2013)
url_list = [url2021_list,url2020_list, url2019_list, url2018_list, url2017_list,url2016_list,url2015_list,
            url2014_list,url2013_list]

In [36]:
sal_url = 'http://www.espn.com/nba/salaries/_/year/2021/page/1'

salary_response = requests.get(sal_url)
salary_page = salary_response.text
salary_soup = BeautifulSoup(salary_page, "lxml")


In [37]:
salary_results = salary_soup.find('div', id='my-players-table').find_all('tr')
salary_dict = {}
for row in salary_results[1:]:
    salary_data = row.find_all('td')
    salary_player = salary_data[0].text
    salary_dict[salary_player]=[i.text for i in salary_data]
salary_dict


{'1': ['1', 'Stephen Curry, PG', 'Golden State Warriors', '$43,006,362'],
 '2': ['2', 'Chris Paul, PG', 'Phoenix Suns', '$41,358,814'],
 '3': ['3', 'Russell Westbrook, PG', 'Washington Wizards', '$41,358,814'],
 '4': ['4', 'James Harden, SG', 'Brooklyn Nets', '$41,254,920'],
 '5': ['5', 'John Wall, PG', 'Houston Rockets', '$41,254,920'],
 '6': ['6', 'Kevin Durant, PF', 'Brooklyn Nets', '$40,108,950'],
 '7': ['7', 'LeBron James, SF', 'Los Angeles Lakers', '$39,219,565'],
 '8': ['8', 'Paul George, SG', 'LA Clippers', '$35,450,412'],
 '9': ['9', 'Klay Thompson, SG', 'Golden State Warriors', '$35,361,360'],
 '10': ['10', 'Mike Conley, PG', 'Utah Jazz', '$34,502,130'],
 'RK': ['RK', 'NAME', 'TEAM', 'SALARY'],
 '11': ['11', 'Kemba Walker, PG', 'Boston Celtics', '$34,379,100'],
 '12': ['12', 'Jimmy Butler, SF', 'Miami Heat', '$34,379,100'],
 '13': ['13', 'Kawhi Leonard, SF', 'LA Clippers', '$34,379,100'],
 '14': ['14', 'Tobias Harris, PF', 'Philadelphia 76ers', '$34,358,850'],
 '15': ['15', '

In [38]:
salary_df = pd.DataFrame(salary_dict).T
salary_df.columns = ['RANK','PLAYER','TEAM','SALARY']

salary_df.drop(salary_df[salary_df['RANK']=='RK'].index, inplace =True)

salary_df.head()


,RANK,PLAYER,TEAM,SALARY
1,1,"Stephen Curry, PG",Golden State Warriors,"$43,006,362"
2,2,"Chris Paul, PG",Phoenix Suns,"$41,358,814"
3,3,"Russell Westbrook, PG",Washington Wizards,"$41,358,814"
4,4,"James Harden, SG",Brooklyn Nets,"$41,254,920"
5,5,"John Wall, PG",Houston Rockets,"$41,254,920"


In [39]:
def get_salary_data(url):
    response = requests.get(url)
    web_page = response.text
    soup = BeautifulSoup(web_page,'lxml')
    salary_results = soup.find('div', id='my-players-table').find_all('tr')
    salary_dict={}
    for row in salary_results[1:]:
        if len(salary_results)==1:
            continue
        salary_data = row.find_all('td')
        salary_player = salary_data[0].text
        salary_dict[salary_player]=[i.text for i in salary_data]
    salary_df = pd.DataFrame(salary_dict).T
    salary_df.columns = ['RANK','PLAYER','TEAM','SALARY']
    salary_df.drop(salary_df[salary_df['RANK']=='RK'].index, inplace =True)
    if len(url) ==52:
        salary_df['YEAR']= url[-12:-8]
    else:
        salary_df['YEAR']= url[-11:-7] 
    return salary_df

In [40]:
test_url = 'http://www.espn.com/nba/salaries/_/year/2021/page/18'
    
response = requests.get(test_url)
web_page = response.text
soup = BeautifulSoup(web_page,'lxml')
salary_results = soup.find('div', id='my-players-table').find_all('tr')
# salary_dict={}
# for row in salary_results[1:]:
#     salary_data = row.find_all('td')
#     salary_player = salary_data[0].text
#     salary_dict[salary_player]=[i.text for i in salary_data]
len(salary_results)

1

In [41]:
salary_df_list = []
for outter_url in url_list:
    for inner_url in outter_url:
        new_salary_df = get_salary_data(inner_url)
        salary_df_list.append(new_salary_df)
salary_df_complete = pd.concat(salary_df_list)
                      
salary_df_complete.info()                        

<class 'pandas.core.frame.DataFrame'>
Index: 4663 entries, 1 to 520
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   RANK    4663 non-null   object
 1   PLAYER  4663 non-null   object
 2   TEAM    4663 non-null   object
 3   SALARY  4663 non-null   object
 4   YEAR    4663 non-null   object
dtypes: object(5)
memory usage: 218.6+ KB


In [42]:
salary_df_complete['SALARY'] = salary_df_complete['SALARY'].str.replace('$','')
salary_df_complete['SALARY'] = salary_df_complete['SALARY'].str.replace(',','')
salary_df_complete.head()



,RANK,PLAYER,TEAM,SALARY,YEAR
1,1,"Stephen Curry, PG",Golden State Warriors,43006362,2021
2,2,"Chris Paul, PG",Phoenix Suns,41358814,2021
3,3,"Russell Westbrook, PG",Washington Wizards,41358814,2021
4,4,"James Harden, SG",Brooklyn Nets,41254920,2021
5,5,"John Wall, PG",Houston Rockets,41254920,2021


In [43]:
sal_name_list = []
for values in salary_df_complete['PLAYER'].values:
    new_value = values.split(',')[0]
    sal_name_list.append(new_value)


In [44]:
salary_df_complete['PLAYER']=sal_name_list

In [45]:
salary_df_complete['SALARY'] = salary_df_complete['SALARY'].astype('int')

In [46]:
salary_df_complete.head()

,RANK,PLAYER,TEAM,SALARY,YEAR
1,1,Stephen Curry,Golden State Warriors,43006362,2021
2,2,Chris Paul,Phoenix Suns,41358814,2021
3,3,Russell Westbrook,Washington Wizards,41358814,2021
4,4,James Harden,Brooklyn Nets,41254920,2021
5,5,John Wall,Houston Rockets,41254920,2021


In [47]:
final_df = pd.merge(final_df,salary_df_complete, on=['PLAYER','YEAR'],how = "left")

final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6797 entries, 0 to 6796
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLAYER          6797 non-null   object 
 1   GAMES           6797 non-null   int64  
 2   MINUTES_PLAYED  6797 non-null   float64
 3   FIELD_GOALS     6797 non-null   float64
 4   FIELD_GOAL_PCT  6797 non-null   float64
 5   3PT_M           6797 non-null   float64
 6   3PT_PCT         6797 non-null   float64
 7   2PT_M           6797 non-null   float64
 8   2PT_PCT         6797 non-null   float64
 9   FT_M            6797 non-null   float64
 10  FT_PCT          6797 non-null   float64
 11  OFF_REB         6797 non-null   float64
 12  DEF_REB         6797 non-null   float64
 13  TOT_REB         6797 non-null   float64
 14  ASSISTS         6797 non-null   float64
 15  STEALS          6797 non-null   float64
 16  BLOCKS          6797 non-null   float64
 17  TO              6797 non-null   f

In [48]:
#drop players we do not have salary information on
final_df.dropna(subset=['SALARY'], inplace = True)


In [49]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3965 entries, 0 to 6714
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLAYER          3965 non-null   object 
 1   GAMES           3965 non-null   int64  
 2   MINUTES_PLAYED  3965 non-null   float64
 3   FIELD_GOALS     3965 non-null   float64
 4   FIELD_GOAL_PCT  3965 non-null   float64
 5   3PT_M           3965 non-null   float64
 6   3PT_PCT         3965 non-null   float64
 7   2PT_M           3965 non-null   float64
 8   2PT_PCT         3965 non-null   float64
 9   FT_M            3965 non-null   float64
 10  FT_PCT          3965 non-null   float64
 11  OFF_REB         3965 non-null   float64
 12  DEF_REB         3965 non-null   float64
 13  TOT_REB         3965 non-null   float64
 14  ASSISTS         3965 non-null   float64
 15  STEALS          3965 non-null   float64
 16  BLOCKS          3965 non-null   float64
 17  TO              3965 non-null   f

### GRAB CONTRACT YEARS

In [50]:
contract_team_list = ['boston-celtics','brooklyn-nets','new-york-knicks','philadelphia-76ers','toronto-raptors',
                     'chicago-bulls','cleveland-cavaliers','detroit-pistons','indiana-pacers','milwaukee-bucks',
                     'atlanta-hawks','charlotte-hornets','miami-heat','orlando-magic','washington-wizards',
                    'denver-nuggets','minnesota-timberwolves','oklahoma-city-thunder','portland-trail-blazers',
                     'utah-jazz','golden-state-warriors','los-angeles-clippers','los-angeles-lakers','phoenix-suns',
                     'sacramento-kings','dallas-mavericks','houston-rockets','memphis-grizzlies','new-orleans-pelicans',
                     'san-antonio-spurs']
contract_url_list=[]

for team in contract_team_list:
    contract_url = 'https://www.spotrac.com/nba/'+team+'/contracts/'
    contract_url_list.append(contract_url)

In [51]:
def get_contract_data(url):
#get html code from webpage
    contract_response = requests.get(url)
    contract_page = contract_response.text
    contract_soup = BeautifulSoup(contract_page,'lxml')
#search for the table rows
    contract_table = contract_soup.find_all('div',class_='teams')[0].find_all('tr')
#populate table data into dict 
    contract_dict = {}
    for row in contract_table[1:]:
        contract_data = row.find_all('td')
        contract_player = contract_data[0].text
        contract_dict[contract_player]=[i.text for i in contract_data]
#turn dict into a df
    contract_df = pd.DataFrame(contract_dict).T
    contract_df.columns = ['PLAYER','POSITION','AGE','EXP','CONTRACT','AVG_SAL','GUARANTEED','EXPIRES']
#split apart contract column to create two new columns for contract length and total    
    contract_len_list = []
    contract_tot_list = []
    for value in contract_df['CONTRACT']:
        value_list= value.split('$')
        string = value_list[0]
        total_year = string[-5:]
        total_year = total_year[0]
        total_amount = string[0:-5:]
        contract_len_list.append(total_year)
        contract_tot_list.append(total_amount)
    contract_df['CONTRACT_LEN']=contract_len_list
    contract_df['CONTRACT_TOT']=contract_tot_list
    return contract_df


In [52]:
contract_df_list=[]
for url in contract_url_list:
    new_df=get_contract_data(url)
    contract_df_list.append(new_df)
contract_df_complete = pd.concat(contract_df_list)

display(contract_df_complete.head())
display(contract_df_complete.info())
    
    
    


,PLAYER,POSITION,AGE,EXP,CONTRACT,AVG_SAL,GUARANTEED,EXPIRES,CONTRACT_LEN,CONTRACT_TOT
TatumJayson Tatum,TatumJayson Tatum,PF,23,4,"1630003005 yr $163,000,300\nRK-MAX\n","$32,600,060","$163,000,300",2026,5,163000300
HorfordAl Horford,HorfordAl Horford,PF,35,14,"1090000004 yr $109,000,000\n","$27,250,000","$97,000,000",2023,4,109000000
BrownJaylen Brown,BrownJaylen Brown,SG,24,5,"1063333344 yr $106,333,334\n","$26,583,334","$102,999,998",2024,4,106333334
FournierEvan Fournier,FournierEvan Fournier,SG,28,9,"780000004 yr $78,000,000\n","$19,500,000",-,2025,4,78000000
SmartMarcus Smart,SmartMarcus Smart,PG,27,7,"519999004 yr $51,999,900\n","$12,999,975","$50,000,000",2022,4,51999900


<class 'pandas.core.frame.DataFrame'>
Index: 404 entries, TatumJayson Tatum  to LandaleJock Landale 
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PLAYER        404 non-null    object
 1   POSITION      404 non-null    object
 2   AGE           404 non-null    object
 3   EXP           404 non-null    object
 4   CONTRACT      404 non-null    object
 5   AVG_SAL       404 non-null    object
 6   GUARANTEED    404 non-null    object
 7   EXPIRES       404 non-null    object
 8   CONTRACT_LEN  404 non-null    object
 9   CONTRACT_TOT  404 non-null    object
dtypes: object(10)
memory usage: 34.7+ KB


None

In [53]:
contract_player_list = []
for string in contract_df_complete['PLAYER']:
    string_list = string.split()
    new_str = string_list[0].replace(string_list[0],string_list[0][len(string_list[1]):])
    full_string = new_str +' '+ string_list[1]
    contract_player_list.append(full_string)
contract_df_complete['PLAYER']=contract_player_list
# contract_player_list




In [54]:
contract_df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Index: 404 entries, TatumJayson Tatum  to LandaleJock Landale 
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PLAYER        404 non-null    object
 1   POSITION      404 non-null    object
 2   AGE           404 non-null    object
 3   EXP           404 non-null    object
 4   CONTRACT      404 non-null    object
 5   AVG_SAL       404 non-null    object
 6   GUARANTEED    404 non-null    object
 7   EXPIRES       404 non-null    object
 8   CONTRACT_LEN  404 non-null    object
 9   CONTRACT_TOT  404 non-null    object
dtypes: object(10)
memory usage: 34.7+ KB


In [55]:
contract_df_complete['EXPIRES']=contract_df_complete['EXPIRES'].replace('-',np.NaN)


contract_df_complete['EXPIRES'].values

array(['2026', '2023', '2024', '2025', '2022', '2022', '2022', '2023',
       '2022', '2023', '2024', '2023', '2024', '2022', '2022', '2023',
       '2023', '2023', '2024', '2024', '2023', '2023', '2022', '2022',
       '2022', '2023', '2022', '2022', '2022', '2024', '2024', '2024',
       '2024', '2023', '2022', '2024', '2024', '2022', '2024', '2022',
       '2024', '2025', '2023', '2023', '2024', '2023', '2024', '2023',
       '2023', '2025', '2023', '2022', '2022', '2022', '2024', '2024',
       '2024', '2025', '2024', '2022', '2022', '2024', '2022', '2022',
       '2022', '2023', '2022', '2022', '2025', '2022', '2022', '2023',
       '2022', '2024', '2024', nan, '2023', '2026', '2022', '2023',
       '2024', '2022', '2023', '2025', '2024', '2023', '2022', '2024',
       '2023', '2023', nan, '2023', '2023', '2022', '2024', '2023',
       '2023', '2022', '2024', '2024', '2022', '2022', '2023', '2023',
       '2024', '2023', '2022', '2024', '2025', '2022', '2023', '2022',
       '2023

In [56]:
#drop values that dont provide salary amount 
contract_df_complete[contract_df_complete['AVG_SAL']=='-']
contract_df_complete['AVG_SAL']=contract_df_complete['AVG_SAL'].replace('-',np.NaN)
contract_df_complete.dropna(subset=['AVG_SAL'], inplace = True)

In [57]:
contract_df_complete['EXPIRES']=contract_df_complete['EXPIRES'].astype('float')
contract_df_complete['CONTRACT_TOT']=contract_df_complete['CONTRACT_TOT'].astype('float')


In [58]:
contract_df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Index: 382 entries, TatumJayson Tatum  to DiengGorgui Dieng 
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PLAYER        382 non-null    object 
 1   POSITION      382 non-null    object 
 2   AGE           382 non-null    object 
 3   EXP           382 non-null    object 
 4   CONTRACT      382 non-null    object 
 5   AVG_SAL       382 non-null    object 
 6   GUARANTEED    382 non-null    object 
 7   EXPIRES       378 non-null    float64
 8   CONTRACT_LEN  382 non-null    object 
 9   CONTRACT_TOT  382 non-null    float64
dtypes: float64(2), object(8)
memory usage: 32.8+ KB


In [59]:
#drop players who signed 1-day contracts

contract_df_complete[contract_df_complete['CONTRACT_LEN']==' ']
contract_df_complete['CONTRACT_LEN']=contract_df_complete['CONTRACT_LEN'].replace(' ',np.NaN)
contract_df_complete.dropna(subset=['CONTRACT_LEN'], inplace = True)


In [60]:
year_list = ['2022','2021','2020','2019','2018','2017','2016']
contract_df_complete.loc['DiakiteMamadi Diakite ','EXPIRES']=2024
contract_df_complete.loc['DiakiteMamadi Diakite ','EXPIRES']=2024

contract_df_complete.loc['MartinKelan Martin ','EXPIRES']=2021
contract_df_complete.loc['BeasleyMalik Beasley ','EXPIRES']=2020

contract_df_complete.loc['NunnKendrick Nunn ','EXPIRES']=2023

In [61]:
contract_df_complete['CONTRACT_LEN']=contract_df_complete['CONTRACT_LEN'].astype('float')

In [62]:
contract_df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Index: 380 entries, TatumJayson Tatum  to DiengGorgui Dieng 
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PLAYER        380 non-null    object 
 1   POSITION      380 non-null    object 
 2   AGE           380 non-null    object 
 3   EXP           380 non-null    object 
 4   CONTRACT      380 non-null    object 
 5   AVG_SAL       380 non-null    object 
 6   GUARANTEED    380 non-null    object 
 7   EXPIRES       380 non-null    float64
 8   CONTRACT_LEN  380 non-null    float64
 9   CONTRACT_TOT  380 non-null    float64
dtypes: float64(3), object(7)
memory usage: 42.7+ KB


In [63]:
contract_df_complete["YEAR_SIGNED"]= contract_df_complete['EXPIRES']-contract_df_complete['CONTRACT_LEN']

In [64]:
contract_df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Index: 380 entries, TatumJayson Tatum  to DiengGorgui Dieng 
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PLAYER        380 non-null    object 
 1   POSITION      380 non-null    object 
 2   AGE           380 non-null    object 
 3   EXP           380 non-null    object 
 4   CONTRACT      380 non-null    object 
 5   AVG_SAL       380 non-null    object 
 6   GUARANTEED    380 non-null    object 
 7   EXPIRES       380 non-null    float64
 8   CONTRACT_LEN  380 non-null    float64
 9   CONTRACT_TOT  380 non-null    float64
 10  YEAR_SIGNED   380 non-null    float64
dtypes: float64(4), object(7)
memory usage: 45.6+ KB


In [65]:
contract_df_complete["YEAR_SIGNED"]=contract_df_complete["YEAR_SIGNED"].astype('int')
contract_df_complete["CONTRACT_LEN"]=contract_df_complete["CONTRACT_LEN"].astype('int')
contract_df_complete["EXPIRES"]=contract_df_complete["EXPIRES"].astype('int')

In [66]:
contract_df_complete = contract_df_complete.drop(['POSITION','AGE','EXP','CONTRACT','AVG_SAL','GUARANTEED'],axis=1)
contract_df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Index: 380 entries, TatumJayson Tatum  to DiengGorgui Dieng 
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PLAYER        380 non-null    object 
 1   EXPIRES       380 non-null    int64  
 2   CONTRACT_LEN  380 non-null    int64  
 3   CONTRACT_TOT  380 non-null    float64
 4   YEAR_SIGNED   380 non-null    int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 27.8+ KB


In [67]:
contract_df_complete['PLAYER']=contract_df_complete['PLAYER'].str.lower()
final_df['PLAYER']=final_df['PLAYER'].str.lower()

In [68]:
final_df_2 = pd.merge(final_df,contract_df_complete, on=['PLAYER'],how = 'left')
final_df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3965 entries, 0 to 3964
Data columns (total 31 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLAYER          3965 non-null   object 
 1   GAMES           3965 non-null   int64  
 2   MINUTES_PLAYED  3965 non-null   float64
 3   FIELD_GOALS     3965 non-null   float64
 4   FIELD_GOAL_PCT  3965 non-null   float64
 5   3PT_M           3965 non-null   float64
 6   3PT_PCT         3965 non-null   float64
 7   2PT_M           3965 non-null   float64
 8   2PT_PCT         3965 non-null   float64
 9   FT_M            3965 non-null   float64
 10  FT_PCT          3965 non-null   float64
 11  OFF_REB         3965 non-null   float64
 12  DEF_REB         3965 non-null   float64
 13  TOT_REB         3965 non-null   float64
 14  ASSISTS         3965 non-null   float64
 15  STEALS          3965 non-null   float64
 16  BLOCKS          3965 non-null   float64
 17  TO              3965 non-null   f

In [69]:
final_df_2 = final_df_2.drop(['TEAM','RANK','TEAM_y'],axis=1)


In [76]:
final_df_3 = final_df_2.dropna(subset=['CONTRACT_LEN'])

In [82]:
final_df_3['YEAR'] = final_df_3['YEAR'].astype('int')



final_df_3 = final_df_3[final_df_3['YEAR']<final_df_3['YEAR_SIGNED']]

In [83]:
final_df_3['EXPERIENCE']=final_df_3['EXPERIENCE'].astype('float')

In [84]:
final_df_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1283 entries, 17 to 3939
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLAYER          1283 non-null   object 
 1   GAMES           1283 non-null   int64  
 2   MINUTES_PLAYED  1283 non-null   float64
 3   FIELD_GOALS     1283 non-null   float64
 4   FIELD_GOAL_PCT  1283 non-null   float64
 5   3PT_M           1283 non-null   float64
 6   3PT_PCT         1283 non-null   float64
 7   2PT_M           1283 non-null   float64
 8   2PT_PCT         1283 non-null   float64
 9   FT_M            1283 non-null   float64
 10  FT_PCT          1283 non-null   float64
 11  OFF_REB         1283 non-null   float64
 12  DEF_REB         1283 non-null   float64
 13  TOT_REB         1283 non-null   float64
 14  ASSISTS         1283 non-null   float64
 15  STEALS          1283 non-null   float64
 16  BLOCKS          1283 non-null   float64
 17  TO              1283 non-null   

In [85]:
final_df_3.describe()

,GAMES,MINUTES_PLAYED,FIELD_GOALS,FIELD_GOAL_PCT,3PT_M,3PT_PCT,2PT_M,2PT_PCT,FT_M,FT_PCT,...,BLOCKS,TO,POINTS,YEAR,EXPERIENCE,SALARY,EXPIRES,CONTRACT_LEN,CONTRACT_TOT,YEAR_SIGNED
count,1283.000000,1283.000000,1283.000000,1283.000000,1283.000000,1283.000000,1283.000000,1283.000000,1283.000000,1283.000000,...,1283.00000,1283.000000,1283.000000,1283.000000,1283.000000,1.283000e+03,1283.000000,1283.000000,1.283000e+03,1283.00000
mean,57.431800,25.957210,4.517615,0.469412,1.088542,0.309882,3.432190,0.513493,2.081450,0.753539,...,0.58940,1.488153,12.202650,2017.743570,4.721746,9.312146e+06,2023.054560,2.716290,4.695139e+07,2020.33827
std,22.016707,7.234967,2.129750,0.068905,0.853213,0.146432,1.902913,0.068563,1.646525,0.114454,...,0.49834,0.844898,6.118694,1.065936,3.082038,8.369693e+06,1.075968,1.231645,5.411475e+07,0.90120
min,1.000000,4.600000,0.400000,0.268000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,1.000000,2017.000000,1.000000,5.982000e+04,2022.000000,1.000000,1.789256e+06,2018.00000
25%,44.000000,20.300000,2.900000,0.424000,0.400000,0.289000,1.900000,0.473000,1.000000,0.706000,...,0.20000,0.900000,7.700000,2017.000000,2.000000,2.328530e+06,2022.000000,2.000000,5.257744e+06,2020.00000
50%,64.000000,27.300000,4.100000,0.459000,1.000000,0.348000,3.100000,0.513000,1.600000,0.776000,...,0.40000,1.300000,11.000000,2017.000000,4.000000,6.006600e+06,2023.000000,3.000000,2.916270e+07,2021.00000
75%,76.000000,31.900000,5.750000,0.502000,1.700000,0.377000,4.700000,0.548000,2.500000,0.836000,...,0.80000,2.000000,15.100000,2019.000000,7.000000,1.457585e+07,2024.000000,4.000000,6.176250e+07,2021.00000
max,82.000000,37.800000,10.900000,0.729000,4.100000,1.000000,9.700000,1.000000,9.200000,1.000000,...,2.90000,5.700000,31.600000,2021.000000,16.000000,3.850648e+07,2026.000000,5.000000,2.282004e+08,2022.00000


In [87]:
final_df_3.to_csv('linreg_final_df.csv',index=False)